# Interpolate the numerical accuracy threshold search results

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
#import re
#import datetime
#import shutil
import qsoconfig
import qsoresult

## Configuration

In [ ]:
data_dir = "example_output"
result_subdir = "ising_chain-3q-nal"
#data_dir = "output/optim_CNOT"
#result_subdir = "nal"
#set comb_coll_fname=None to collate separately
comb_coll_fname = None
#result_subdir = "5qubit-chain-ising_equal-xy_ctrl-cNOT1-fet1e-1-na2e-2"
param_file_pat = "params*.ini"
coll_file_pat = "nal_collate*.txt"
interp_fname = "interp.dat"
plot_fname = "interplot.png"
verbosity = 1



## Loading the result data

In [ ]:
if '~' in data_dir:
    data_dir = os.path.expanduser(data_dir)
elif not os.path.abspath(data_dir):
    # Assume relative path from cwd given
    data_dir = os.path.join(os.getcwd(), data_dir)

res_dir = os.path.join(data_dir, result_subdir)

print("Results dir:\n{}".format(res_dir))
if not os.path.isdir(res_dir):
    raise RuntimeError("Results dir not valid")

# look for files to read existing results
coll_na = {}
na_list = []

if verbosity > 0:
    print("Searching in:\n{}".format(res_dir))
    print("Looking for collation file matching {}\n".format(coll_file_pat))
files = glob.glob(os.path.join(res_dir, coll_file_pat))

if len(files) == 0:
    print("No collation file to process")

plot_fpath = os.path.join(res_dir, plot_fname)

# Take the most recent file, assume last in list
collf = sorted(files)[-1]
if verbosity > 0:
    print("Loading collation from file:\n{}\n".format(collf))
reslist = qsoresult.MultiRepResult.load_from_txt(collf)
for na_res in reslist:
    na_list.append(na_res.numer_acc)
    coll_na[na_res.numer_acc] = na_res

na_arr = np.array(sorted(na_list))
succ_props = []
num_iters = []
for na in na_arr:
    na_res = coll_na[na]
    succ_props.append(float(na_res.num_primary_success) / na_res.num_res)
    num_iters.append(na_res.mean_iter_primary_success)
    fid_err_targ = na_res.fid_err_targ

succ_arr = np.array(succ_props)
iter_arr = np.array(num_iters)
na_lim = na_arr[-1]*1.2

# Look for previous interp results file
interp_fpath = os.path.join(res_dir, interp_fname)
if os.path.isfile(interp_fpath):
    if verbosity > 0:
        print("Loading interp params from:\n{}\n".format(interp_fpath))
    data = np.loadtxt(interp_fpath)
    # Note num_qubits only added to the file 2018-04-13
    # interp files from before that will need number of qubits inserting
    nq = int(data[0])
    na_lb = data[1]
    na_ub = data[2]
    try:
        succ_thresh = data[3]
    except:
        succ_thresh = 0.5
else:
    # No interp results file, look for a parameter file
    param_pat = os.path.join(res_dir, param_file_pat)
    if verbosity > 0:
        print("Looking for parameter file matching:\n{}".format(param_pat))
    files = glob.glob(param_pat)
    n_files = len(files)
    if n_files == 0:
        print("NO PARAMETER FILES FOUND!")
        raise RuntimeError("No parameter file")
    print("Loading configuration from:\n{}".format(files[0]))
    optim = qsoconfig.gen_config(files[0])
    dyn = optim.dynamics
    tc = optim.termination_conditions
    fid_comp = optim.dynamics.fid_computer
    nq = dyn.num_qubits
    if not fid_comp.numer_acc_exact:
        fid_comp.st_numer_acc = round_sigfigs(
                fid_comp.st_numer_acc*fid_err_targ/tc.fid_err_targ, 6)
        fid_comp.end_numer_acc = round_sigfigs(
                fid_comp.end_numer_acc*fid_err_targ/tc.fid_err_targ, 6)

    na_lb = fid_comp.st_numer_acc
    na_ub = fid_comp.end_numer_acc
    succ_thresh = 0.5

